In [ ]:
#-----------------Zero-ing Script-----------------------
from IPython.display import clear_output
import numpy as np
import serial
import struct
import math
import time
serialPort = serial.Serial(port = "COM4",baudrate=115200, bytesize=8,stopbits=serial.STOPBITS_ONE)
Theta = np.array([0,0,0,0,0])
for i in np.arange(1,6,1):
    Time = Theta*0.01031 + 0.544
    RB = abs(Time*24e3/16)
    RB_array = np.array(RB, dtype = np.single) # this converts the array values to float values.
    RB_binary = RB_array.tobytes()
    voyager = np.array([160,i,RB_binary[(i-1)*4],RB_binary[(i-1)*4+1],RB_binary[(i-1)*4+2],RB_binary[(i-1)*4+3],176,177], dtype = np.uint8)
    serialPort.write(voyager.data)       
    print(i)
    #readserial = (serialPort.read(size = 8))
    #distarray = [readserial[2],readserial[3],readserial[4],readserial[5]]
    #distance = struct.unpack('f',bytes(distarray))      
    
print("Closing Serial Port")
serialPort.close()

In [ ]:
#---------Testing importing of text file values and readings---------
import keyboard
import numpy as np
import time
counter = 0
while True:
    counter = counter + 1
    print("\n Iteration ", counter, "\n")
    fcoordinates = open("C:/ODA/models/research/object_detection/Coordinates.txt","r")
    fprobability = open("C:/ODA/models/research/object_detection/Probability.txt","r")
    fclass = open("C:/ODA/models/research/object_detection/Class.txt","r")
    classes = fclass.read()
    coordinates = fcoordinates.readlines()
    probabilty = float(fprobability.read())
    print("A ",classes," has been detected at the following coordinates:")
    print("probability: ",probabilty)
    i = 0
    value = []
    for k in coordinates:
        i = i + 1
        if (i == 1) or (i == 3):
            value.append(np.round((float(k) * 480),2))
        else:
            value.append(np.round((float(k) * 640),2))
            
    print("The coordinates: ", value)
    
    # all values in cm
    distance  = 35
    xmin = value[1]
    xmax = value[3]
    width = xmax - xmin
    pixelmovement = (320 + width*0.5) - xmax
    pixelratio = pixelmovement/320
    R1 = np.tan(np.radians(20.5)) * distance
    realmovement = R1*pixelratio
    rotation = np.degrees(np.arctan(realmovement/distance))
    #print(" The base needs to rotate a value of ",rotation," degrees")
    fclass.close()
    fcoordinates.close()
    if classes == "person\n":
        if pixelmovement < 0:
            print("bottle is detected, and must move has to move ", abs(pixelmovement), " pixels right into position \n")
            print("This is equivalent to ",abs(realmovement), "cm right in real space")
            print("inducing a rotation of {} degrees".format(rotation))
        else:
            print("bottle is detected, and must move has to move ", abs(pixelmovement), " pixels left into position")
            print("This is equivalent to ",abs(realmovement), "cm left in real space")
            print("inducing a rotation of {} degrees".format(rotation))
        break
    time.sleep(1)
    if keyboard.is_pressed('q'):
        print("key pressed")
        break
        
    if counter > 10:
        print("We have reached 10 iterations for updating coordinates")
        break
   

In [ ]:
# ----------------------------------------------Sonar Testing-----------------------------------------------------------
from IPython.display import clear_output
import numpy as np
import keyboard
import serial
import struct
import math
import time
serialPort = serial.Serial(port = "COM4",baudrate=115200, bytesize=8,stopbits=serial.STOPBITS_ONE)
count = 0
while True:
    clear_output()
    count = count + 1
    print("\nIteration ", count, "\n")
    Theta = np.array([0,0,90,0,0])
    Time = Theta*0.01031 + 0.544
    RB = abs(Time*24e3/16)
    RB_array = np.array(RB, dtype = np.single) # this converts the array values to float values.
    RB_binary = RB_array.tobytes()
    for i in np.arange(1,6,1):
        voyager = np.array([160,i,RB_binary[(i-1)*4],RB_binary[(i-1)*4+1],RB_binary[(i-1)*4+2],RB_binary[(i-1)*4+3],176,177], dtype = np.uint8)
        serialPort.write(voyager.data)       
        time.sleep(0.25)
        readserial = (serialPort.read(size = 8))
        distarray = [readserial[2],readserial[3],readserial[4],readserial[5]]
        distance = struct.unpack('f',bytes(distarray))
        
    if distance[0] > 1000:
        print("Sonar does not pickup anything")
    else:
        print("This is the current distance with a delay of 1 seconds ", np.round(distance[0],2))
        
    time.sleep(5)
    
    if keyboard.is_pressed('q'):
        clear_output()
        print("exit")
        break
serialPort.close()

In [ ]:
import numpy as np
Theta = np.array([0,75,100,135,0])
OFFSET2 = 35
Theta2 = 75
l1 = 0.13
l2 = 0.13
l3 = 0.15
h1 = 0.07
phi = -60
df3 = -l1 * np.sin(np.radians(Theta2 - OFFSET2 - 90)) + l2 + l3 * np.cos(np.radians(abs(phi)))
h3 = h1 + l1 * np.cos(np.radians(Theta2 - OFFSET2 - 90)) - l3 * np.sin(np.radians(abs(phi)))
ZeroDistance = df3 + h3/(np.tan(np.radians((180 - (90 - abs(phi)) - 69.95))))
print("The zero distance detected from the camera geometry at 480 pixels of teh y axis is: ", np.round(100*ZeroDistance,2), "cm")


In [ ]:
import numpy as np
import struct
import time
Theta = np.array([0, 150, 3, 177,   0])
Time = Theta*0.01031 + 0.544
RB = abs(Time*24e3/16)
RB_array = np.array(RB, dtype = np.single) 
RB_binary = RB_array.tobytes()
for i in np.arange(1,6,1):
            voyager = np.array([160,i,RB_binary[(i-1)*4],RB_binary[(i-1)*4+1],RB_binary[(i-1)*4+2],RB_binary[(i-1)*4+3],176,177], dtype = np.uint8)
            serialPort.write(voyager.data) 
            time.sleep(0.1)
            readserial = (serialPort.read(size = 8))
            distarray = [readserial[2],readserial[3],readserial[4],readserial[5]]
            distance = struct.unpack('f',bytes(distarray)) 
print('distance', distance[0] + 6.75)